In [1]:
import pandas as pd
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_colwidth', 355)
pd.set_option('display.max_rows', 100)

In [2]:
#df = pd.read_csv('datasets/b3_stocks_1994_2020.csv')
df = pd.read_csv('../b3data/b3_stocks_1994_2020.csv')

### IBOV Ações - Exploração inicial

In [3]:
df.head()

,datetime,ticker,open,close,high,low,volume
0,1994-07-04,ACE 3,48.00,48.00,48.00,47.00,46550.0
1,1994-07-04,ALP 3,155.27,156.00,156.00,155.27,163405.8
2,1994-07-04,ALP 4,131.00,131.00,131.00,131.00,6550.0
3,1994-07-04,IBP 6,600.00,600.00,600.00,600.00,7800.0
4,1994-07-04,AQT 4,0.89,0.99,0.99,0.85,13137.0


In [4]:
df.shape

(1883203, 7)

### 

In [5]:
# Convertendo string para datetime
df['datetime'] = pd.to_datetime(df['datetime'])

In [6]:
# Normalizando os valores
df_norm = pd.DataFrame(MinMaxScaler().fit_transform(df), 
                      index = df.index, 
                      columns = df.columns)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [ ]:
# Diferença de abertura e fechamento em porcentagem
df['delta_open'] = df.apply(lambda x: (abs(x['open'] - x['close']) / x['open']) * 100, axis=1)

In [ ]:
sns.boxplot(x=df['delta_open'])

In [ ]:
df['delta_high'] = df.apply(lambda x: (abs(x['high'] - x['close'])), axis=1)

In [ ]:
df.head()

In [ ]:
def calc_delta_side(row):
    delta = row['open'] - row['close']
    if delta > 1:
        return 1
    elif delta == 0:
        return 0
    else:
        return -1

df['delta_side'] = df.apply(lambda row: calc_delta_side(row), axis=1)

In [ ]:
df.head()

In [ ]:
sns.relplot(x="delta_open", y="delta_high", data=df);

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [ ]:
# ZOOM NOS NORMAIS
# sns.relplot(x="delta_open", y="delta_high", data=df.loc[df['delta_high'] < 1000])

In [ ]:
df.loc[df['delta_high'] > 1000].head(100)

## Usaremos dados depois de 95 por causa do plano real
Deltas extramamente altas levantaram a suspeita

In [ ]:
df = df.loc[df['datetime'] > '1995-01-01']

In [ ]:
sns.relplot(x="delta_open", y="delta_high", data=df);

In [ ]:
df.loc[df['delta_high'] > 6000].head(1)

### vamos recortar essa observação que esta extremamente descolada

In [ ]:
df = df.loc[df['delta_high'] < 6999]

In [ ]:
sns.relplot(x="delta_open", y="delta_high", data=df)

In [ ]:
sns.boxplot(x=df['delta_high'])

In [ ]:
df.loc[df['delta_high'] > 600].head(10)

## Vamos usar apenas dados desse seculo
nao conseguimos achar esssas acoes antigas

In [ ]:
df = df.loc[df['datetime'] > '2000-01-01']

In [ ]:
sns.relplot(x="delta_open", y="delta_high", data=df)

In [ ]:
df.loc[df['delta_high'] > 600].head(10)

In [ ]:
sns.lineplot(x='datetime', y='close', data=df.loc[(df['ticker'] == 'AMZO34') & (df['datetime'] > '2020-11-01') & (df['datetime'] > '2020-11-01')])

In [ ]:
df.loc[(df['ticker'] == 'AMZO34') & (df['datetime'] > '2020-11-01') & (df['datetime'] > '2020-11-01')]

### Seriam esses dois dados ruido?

In [ ]:
df.loc[df['delta_open'] > 10].head(10)

### resolvemos tirar o delta_high pois nao traz informação util

In [ ]:
df = df.drop(['delta_high', 'volume'], axis=1)

### LOF

In [ ]:
def get_LOF_scores(df, n_neighbors=10, contamination=0.05):
    np.random.seed(42)

    # fit the model for outlier detection (default)
    clf = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    # use fit_predict to compute the predicted labels of the training samples
    # (when LOF is used for outlier detection, the estimator has no predict,
    # decision_function and score_samples methods).
    y_pred = clf.fit_predict(df)

    X_scores = clf.negative_outlier_factor_
    
    df['LOF_score'] = X_scores
    df['LOF_predictions'] = y_pred
    
    return df

In [ ]:
scores = get_LOF_scores(df_test, n_neighbors=10, contamination=0.3)

In [ ]:
def show_2D_outliers(df, x, y, scores, title = ''):

    normalized = (df[scores].max() - df[scores]) / (df[scores].max() - df[scores].min())
    
    t = "Outlier Scores"
    if title: t=t+": "+title
    
    fig, ax = plt.subplots(figsize=(8, 6))

    plt.title(t)
    plt.scatter(x=x, y=y, color='k', s=3., label='Data points', data=df)
    # plot circles with radius proportional to the outlier scores

    plt.scatter(x=x, y=y, s=1000 * normalized, edgecolors='r',
                facecolors='none', label='Outlier scores', data=df)
    plt.axis('tight')
    # plt.xlim((-5, 5))
    # plt.ylim((-5, 5))
    # plt.xlabel("prediction errors: %d" % (n_errors))
    legend = plt.legend(loc='upper left')
    legend.legendHandles[0]._sizes = [10]
    legend.legendHandles[1]._sizes = [20]
    plt.show()

In [ ]:
show_2D_outliers(scores, x = 'open', y = 'delta_open', scores = 'LOF_score', title = 'Dia Semana')

In [ ]:
scores